In [1]:
import os
import cv2
import yaml
import random
import numpy as np

In [2]:
# 파일 위치가 한글 경로가 존재할 때 사용함(없다면 제외하고 해도 상관 없음)
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None

# 파일 위치가 한글 경로가 존재할 때 사용함(없다면 제외하고 해도 상관 없음)
def imwrite(filename, img, params=None):
    try:
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params)
        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f)
            return True
        else:
            return False
    except Exception as e:
        print(e)
        return False

In [3]:
def img_preprocessing(img_path):
    # 한글 경로로 되어 있지 않을 때 1번, 한글 경로로 되어 있다면 2번을 사용함.
    # 1번
    #re_img = cv2.imread(img_path)
    # 2번
    img = imread(img_path)
    
    #전처리 부분
    # 여기서 416,416으로 이미지 크기를 조정함.
    # 이밖에 모자이크, 흑백 등의 전처리를 추가하고 싶다면 자유롭게 할 것.
    re_img = cv2.resize(img, (416, 416))
    return re_img

In [4]:
def save_files(file_name, Img_path, Change_img_path, Label_path, Change_label_path):
    # A 폴더/train/images에 이미지 넣기, A 폴더/train/labels에 라벨 넣기
    for file in file_name:
        # 이미지 읽고 전처리 후 저장
        img_path = os.path.join(Img_path,file+".png")
        T_img_path = os.path.join(Change_img_path,file+".png")
        p_img = img_preprocessing(img_path)
        # 저장할 이미지 경로에 한글이 없다면 1번, 있다면 2번을 실행한다.
        # 1번
        #cv2.imwrite(Train_img_path,p_img)
        # 2번
        imwrite(T_img_path,p_img)

        # 라벨 읽고 저장
        label_path = os.path.join(Label_path, file+".txt")
        T_label_path = os.path.join(Change_label_path, file+".txt")
        f = open(label_path, 'r')
        h = open(T_label_path,'w')
        data = f.read()
        h.write(data)
        f.close()
        h.close()

In [5]:
# 이미지 저장 폴더(All_Data)에서 이미지 리스트 불러오기
# 이 코드도 1번 코드와 동일한 위치(편의 상 A폴더라고 지칭함)에 있음
path = os.getcwd()
Img_path = os.path.join(path,"All_Data")
file_list = os.listdir(Img_path)
# 이미지 파일 이름 끝이 .png로 끝나므로 .png로 split 후 이름만 가져온다.
file_name_list = [i.split(".png")[0] for i in file_list]

In [6]:
# 랜덤으로 데이터 섞은 후 train, validation 나누기(8:2 비율)
random.shuffle(file_name_list)
Split_Num = int(len(file_name_list) * 0.8)
Train_file_name = file_name_list[:Split_Num]
Valid_file_name = file_name_list[Split_Num:]

In [7]:
# Train 이미지는 A 폴더/train/images 폴더에, Labeling된 값들은 A 폴더/train/labels 폴더에 넣는다.
# Validation 이미지는 A 폴더/valid/images 폴더에, Labeling된 값들은 A 폴더/valid/labels 폴더에 넣는다.
# 이를 위해서 각각 폴더가 만들어있지 않으면 다음을 실행한다.
path = os.getcwd()

pre_img_file_path = os.path.join(path,"All_Data")
pre_label_file_path = os.path.join(path,"All_Data_label")

Train_path = os.path.join(path,"train")
Valid_path = os.path.join(path,"valid")

Train_img_path = os.path.join(Train_path,"images")
Train_label_path = os.path.join(Train_path,"labels")
Valid_img_path = os.path.join(Valid_path,"images")
Valid_label_path = os.path.join(Valid_path,"labels")

path_list = [Train_path,Valid_path,Train_img_path,Train_label_path,Valid_img_path,Valid_label_path]

for p in path_list:
    if not os.path.isdir(p):
        os.mkdir(p)

# 파일 저장
save_files(Train_file_name, pre_img_file_path, Train_img_path, pre_label_file_path, Train_label_path)    
save_files(Valid_file_name, pre_img_file_path, Valid_img_path, pre_label_file_path, Valid_label_path)    

In [8]:
# train 위치, valid 위치 설정하기
# nc = classes num(여기서는 0~5까지 6개이다.)
# names = class 이름

with open(os.path.join(pre_label_file_path,"classes.txt"),'r') as f:
    data = f.read()
    classes = data
names = classes.split("\n")[:-1]
nc = len(names)
yaml_data = {'train': Train_img_path, 'val': Valid_img_path, 
          'nc': nc, 'names': names}
yaml_path = os.path.join(path,'data.yaml')
with open(yaml_path,'w') as f:
    yaml.dump(yaml_data,f)
# 잘 저장되었는 지 확인하기
with open(yaml_path,'r') as f:
    yaml_file = yaml.load(f, Loader=yaml.FullLoader)
    display(yaml_file)

{'names': ['Finger_0',
  'Finger_1',
  'Finger_2',
  'Finger_3',
  'Finger_4',
  'Finger_5'],
 'nc': 6,
 'train': 'C:\\Users\\USER\\Desktop\\수\\Yolo\\정리\\train\\images',
 'val': 'C:\\Users\\USER\\Desktop\\수\\Yolo\\정리\\valid\\images'}